# Ejercicio: Problema de la mochila

Alumno: Jorge Andrés Mármol Rivera

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## Preliminares

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/8- Curso de extensión en computación cuántica/Modulo5

In [ ]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.9/158.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
!dwave config create

Using the simplified configuration flow.
Try 'dwave config create --full' for more options.

Updating existing configuration file: /root/.config/dwave/dwave.conf
Available profiles: defaults
Updating existing profile: defaults
Solver API token [skip]: DEV-f9475f07064074a195542d4109ed8d65b8f015b8
Configuration saved.


## Resolución

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
import numpy as np

Función para leer acorde al formato indicado

In [ ]:
def load_knapsack_instance(filename):
    with open(filename, 'r') as f:
        data = list(map(float, f.read().split()))

    num_variables = int(data[0])
    num_constraints = int(data[1])
    optimal_value = data[2]
    values = data[3:3 + num_variables]
    constraint_coefficients = data[3 + num_variables:3 + num_variables + num_variables * num_constraints]
    rhs_constraints = data[3 + num_variables + num_variables * num_constraints:]
    constraint_coefficients = np.array(constraint_coefficients).reshape((num_constraints, num_variables))

    return num_variables, num_constraints, values, constraint_coefficients, rhs_constraints, optimal_value


Función para construir el qubo

In [ ]:
def build_qubo(num_variables, num_constraints, values, constraint_coefficients, rhs_constraints, penalty):
    qubo = {}

    for i in range(num_variables):
        qubo[(i, i)] = -values[i]

    for j in range(num_constraints):
        for i in range(num_variables):
            qubo[(i, i)] += penalty * (constraint_coefficients[j, i] ** 2)
            for k in range(i + 1, num_variables):
                qubo[(i, k)] = qubo.get((i, k), 0) + penalty * 2 * constraint_coefficients[j, i] * constraint_coefficients[j, k]

    for j in range(num_constraints):
        for i in range(num_variables):
            qubo[(i, i)] -= 2 * penalty * constraint_coefficients[j, i] * rhs_constraints[j]

    return qubo

Función para resolver el QUBO

In [ ]:
def solve_qubo(qubo,num_variables,values):
    sampler = EmbeddingComposite(DWaveSampler())

    result = sampler.sample_qubo(qubo, num_reads=100)

    best_solution = result.first.sample
    best_energy = result.first.energy

    total_value = sum(values[i] for i in range(num_variables) if best_solution[i] == 1)

    return result, best_solution, total_value, best_energy

Función que computa los resultados en base al nombre del archivo

In [ ]:
def solve_knapsack(filename, penalty=10):
    num_variables, num_constraints, values, constraint_coefficients, rhs_constraints, optimal_value = load_knapsack_instance(filename)

    qubo = build_qubo(num_variables, num_constraints, values, constraint_coefficients, rhs_constraints, penalty)
    print(qubo)
    result, best_solution, total_value, best_energy = solve_qubo(qubo, num_variables, values)

    print("\nReal optimal value:")

    print(optimal_value)

    print("\nResults from the quantum annealer:")

    print("Value: ", total_value)

    print("Best Solution: ", best_solution)


Iteramos en los experimentos

In [ ]:
for i in range(1,8):

  filename = f'md-knap{i}.txt'
  print(f"""
***************************************

Experimento para md-knap{i}.txt

***************************************
  """)
  try:
    solve_knapsack(filename)
  except Exception as e:
    print(e)
    print("No se pudo resolver el problema")


***************************************

Experimento para md-knap1.txt
  
***************************************
  
{(0, 0): -43710.0, (1, 1): -70500.0, (2, 2): -68220.0, (3, 3): -213910.0, (4, 4): -93860.0, (5, 5): -153500.0, (0, 1): 8040.0, (0, 2): 7520.0, (0, 3): 36000.0, (0, 4): 10640.0, (0, 5): 19040.0, (1, 2): 12800.0, (1, 3): 65640.0, (1, 4): 16240.0, (1, 5): 33280.0, (2, 3): 57740.0, (2, 4): 16720.0, (2, 5): 30600.0, (3, 4): 79240.0, (3, 5): 159740.0, (4, 5): 44080.0}

Real optimal value:
3800.0

Results from the quantum annealer:
Value:  4300.0
Best Solution:  {0: 0, 1: 1, 2: 1, 3: 0, 4: 1, 5: 1}

***************************************

Experimento para md-knap2.txt
  
***************************************
  
{(0, 0): -3379600.1, (1, 1): -785340.5, (2, 2): -5788800.0, (3, 3): -12722850.0, (4, 4): -317778.6, (5, 5): -627348.7, (6, 6): -4215882.0, (7, 7): -8427950.0, (8, 8): -2825042.5, (9, 9): -2204287.0, (0, 1): 95600.0, (0, 2): 670000.0, (0, 3): 1866000.0, (0, 4): 40800.